In [1]:
import numpy as np
import datanewn as dt
import os
from torch.utils.data import DataLoader
from skimage.filters import sobel
from skimage import color
from skimage.io import imsave

In [ ]:
csvFilePath = dt.get_csv_path()
print(csvFilePath)
transformed_dataset = dt.HE_SHG_Dataset(csv_file=csvFilePath,
                                               transform=dt.Compose([                                              
                                               dt.Rescale(96),                                     
                                               dt.Normalize(),
                                               dt.ToTensor()
                                           ]))
# TODO: change the normalization parameters
dataloader = DataLoader(transformed_dataset, batch_size=50,
                        shuffle=True, num_workers=0)
print(len(transformed_dataset))

In [ ]:
def save_batch(he_batch, shg_batch, iteration):
    cwd = os.getcwd();
    for index, he_img, shg_img in enumerate(zip(he_batch, shg_batch))
        shgpath = os.path.join(cwd, 'Screened', 'SHG', str(iteration)+str(index)+".png")
        hepath = os.path.join(cwd, 'Screened', 'HE', str(iteration)+str(index)+".png")
        imsave(shgpath, shg_img)
        imsave(hepath, he_img)

def avg_edge_index(img_batch):
    batch_edge = np.zeros((img_batch.shape(0)))
    for iteration, img in enumerate(img_batch):
        hsv = color.rgb2hsv(img)
        v = 1 - hsv[:, :, 2]
        edge_sobel = sobel(img)
        avg = edge_sobel / sum(v)
        batch_edge[iteration] = avg
    return np.argsort(batch_edge)

def rank_img(he_batch, shg_batch, index):
    size = shg_batch.shape(0)
    valid = round(size / 1.5)
    he_batch = he_batch[index]
    shg_batch = shg_batch[index]
    he_batch = he_batch[0:valid]
    shg_batch = shg_batch[0:valid]
    return he_batch, shg_batch
    

In [ ]:
def screen():
    for iteration, batch in enumerate(dataloader):
        he_batch, shg_batch = batch['input'], batch['output']
        he_batch = he_batch.numpy().transpose((1, 2, 0))
        shg_batch = shg_batch.numpy().transpose((1, 2, 0))
        index = avg_edge_index(he_batch)
        rank_img(he_batch, shg_batch, index)
        save_batch(he_batch, shg_batch, iteration)